In [1]:
import pandas as pd
import numpy as np
import sys, zipfile, os, time
import matplotlib.pyplot as plt
import glob
import datetime
import urllib3
import cdsapi
import xarray
import shutil
import warnings
import pickle
import h5py
import netCDF4
warnings.filterwarnings('ignore')

sys.path.append(os.getcwd()+'/../cds-backend/code/')
import cds_eua3 as eua

In [2]:
def get_adj(station):
    import datetime
    """Compare created adjustment to downloaded adjustment
    """
    date0 = datetime.datetime.strptime('1900-01-01', "%Y-%m-%d")

    file = glob.glob('./Humidity_adjustments/files/downloaded/downloaded_'+station+'/dest_*_relative_humidity_adjusted.nc')[0]
    data = eua.CDMDataset(file).to_dataframe()
    daydata = data[data.time.dt.hour > 6][data.time.dt.hour <= 18]
    nightdata = data[data.time.dt.hour <= 6].append(data[data.time.dt.hour > 18])
    
#     c = cdsapi.Client()
#     r = c.retrieve('insitu-comprehensive-upper-air-observation-network',
#                    {'variable': 'relative_humidity',
#                     'optional':['humidity_bias_estimate'],
#                     'asdf':'asdfadsf',
#                     'statid': station,
#                     'pressure_level':[10,20,30,50,70,100,150,200,250,300,400,500,700,850,925,1000]
#                    }
#                   )
#     r.download(target='download.zip')
#     assert os.stat('download.zip').st_size == r.content_length, "Downloaded file is incomplete"
#     z = zipfile.ZipFile('download.zip')
#     z.extractall(path='./tocompare/hur/cds_'+station)
#     z.close()
#     cdsfile = glob.glob('./tocompare/hur/cds_'+ station +'/*.nc')
    
    data=eua.vm_request_wrapper( {'variable': 'relative_humidity',
                                  'optional':['humidity_bias_estimate'],
                                  'statid': station,
                                  'newdl':'000111000',
                                  'pressure_level':[1000,2000,3000,5000,7000,10000,15000,20000,25000,30000,40000,50000,70000,85000,92500,100000]
                                 }, overwrite=True, vm_url='http://srvx8.img.univie.ac.at:8000')
    
    cdsdata=data.to_dataframe()
    daycdsdata = cdsdata[cdsdata.time.dt.hour > 6][cdsdata.time.dt.hour <= 18]
    nightcdsdata = cdsdata[cdsdata.time.dt.hour <= 6].append(cdsdata[cdsdata.time.dt.hour > 18])
    
    return daydata, nightdata, daycdsdata, nightcdsdata

In [3]:
def plt_adj(cdata, adata, plev, station, time, adj, save):
    cdata = cdata[cdata.plev == plev*100]
    adata = adata[adata.plev == plev*100]
    fig, ax = plt.subplots()
    ax.scatter(cdata['time'], cdata.humidity_bias_estimate, label='CDS', s=50)
    ax.scatter(adata['time'], adata.hur_bias_estimate, label='Calc', s=20)

    ax.set(xlabel='time', ylabel='Adjustment: '+adj,
           title= str(station)+' Humidity '+str(plev)+' hPa '+time)
    ax.legend()
    ax.grid()
    
    if save:
        fig.savefig(str(station)+'_Humidity_'+str(plev)+'_hPa_'+time+'.png')
    plt.show()
    return

In [5]:
# DOWNLOAD DATA FROM CDS:
home = os.getcwd()
%run gethum.py # downloading just: ['0-20001-0-11035', '0-20001-0-10393', '0-20001-0-70219', '0-20000-0-91413', '0-20000-0-35229', '0-20000-0-68994']
os.chdir(home)

Humidity_adjustments/files


In [4]:
files = glob.glob('Humidity_adjustments/files/downloaded/*/*relative_humidity.nc')
display(files)

['Humidity_adjustments/files/downloaded/downloaded_10393/dest_0-20001-0-10393_relative_humidity.nc',
 'Humidity_adjustments/files/downloaded/downloaded_11035/dest_0-20001-0-11035_relative_humidity.nc',
 'Humidity_adjustments/files/downloaded/downloaded_35229/dest_0-20000-0-35229_relative_humidity.nc',
 'Humidity_adjustments/files/downloaded/downloaded_68994/dest_0-20000-0-68994_relative_humidity.nc',
 'Humidity_adjustments/files/downloaded/downloaded_70219/dest_0-20000-0-70219_relative_humidity.nc',
 'Humidity_adjustments/files/downloaded/downloaded_91413/dest_0-20000-0-91413_relative_humidity.nc']

In [10]:
# RUN ADJUSTMENTS
import subprocess
for i in files:
    exc = ['./raso_adj_cdm_v1.py', '-f', './'+i, '--humidity', '--verbose']
    list_files = subprocess.run(exc)


In [11]:
exc

['./raso_adj_cdm_v1.py',
 '-f',
 './Humidity_adjustments/files/downloaded/downloaded_91413/dest_0-20000-0-91413_relative_humidity.nc',
 '--humidity',
 '--verbose']

In [19]:
station = '70219'
dd, nd, dcd, ncd = get_adj(station)
# plt_adj(dcd, dd, 100, station, 'day', 'humidity_bias_estimate', save=True)


In [21]:
data=eua.vm_request_wrapper( {'variable': 'relative_humidity',
                                  'optional':['humidity_bias_estimate'],
                                  'statid': '70219',
                                  'newdl':'000111000',
                                  'pressure_level':[1000,2000,3000,5000,7000,10000,15000,20000,25000,30000,40000,50000,70000,85000,92500,100000]
                                 }, overwrite=True, vm_url='http://srvx8.img.univie.ac.at:8000')

In [24]:
data.humidity_bias_estimate[:]

array([[b'0', b'0', b'0', ..., b'0', b'0', b'0'],
       [b'0', b'0', b'0', ..., b'0', b'0', b'1'],
       [b'0', b'0', b'0', ..., b'0', b'0', b'2'],
       ...,
       [b'0', b'0', b'0', ..., b'9', b'5', b'9'],
       [b'0', b'0', b'0', ..., b'9', b'6', b'0'],
       [b'0', b'0', b'0', ..., b'9', b'6', b'1']], dtype='|S1')

In [26]:
file = glob.glob('/raid60/scratch/leo/scratch/converted_v6/*70219*')[0]
data = eua.CDMDataset(file)

In [30]:
data.advanced_homogenisation

advanced_homogenisation:

RAOBCORE_bias_estimate____________________________ : : (75429178,)
RASE_bias_estimate________________________________ : : (75429178,)
RICH_bias_estimate________________________________ : : (75429178,)
RISE_bias_estimate________________________________ : : (75429178,)
index_____________________________________________ : : (75429178,)
wind_bias_estimate________________________________ : : (75429178,)

In [20]:
dcd

,humidity_bias_estimate,hur,lat,lon,plev,report_id,station_id,time,trajectory_label
3,0000000003,0.660,60.779999,-161.720001,50000.0,60000000001,"BETHEL/BETHEL AIRPORT, AK",1948-01-01 15:00:00,nnnnnnnnnnn
4,0000000004,0.750,60.779999,-161.720001,70000.0,60000000001,"BETHEL/BETHEL AIRPORT, AK",1948-01-01 15:00:00,nnnnnnnnnnn
5,0000000005,0.730,60.779999,-161.720001,85000.0,60000000001,"BETHEL/BETHEL AIRPORT, AK",1948-01-01 15:00:00,nnnnnnnnnnn
9,0000000009,0.440,60.779999,-161.720001,50000.0,60000000003,"BETHEL/BETHEL AIRPORT, AK",1948-01-02 15:00:00,nnnnnnnnnnn
10,0000000010,0.690,60.779999,-161.720001,70000.0,60000000003,"BETHEL/BETHEL AIRPORT, AK",1948-01-02 15:00:00,nnnnnnnnnnn
...,...,...,...,...,...,...,...,...,...
395957,0000395957,0.655,60.785000,-161.839996,50000.0,30000060066,"BETHEL/BETHEL AIRPORT, AK",2021-05-31 12:00:00,nnnnnnnnnnn
395958,0000395958,0.524,60.785000,-161.839996,70000.0,30000060066,"BETHEL/BETHEL AIRPORT, AK",2021-05-31 12:00:00,nnnnnnnnnnn
395959,0000395959,0.842,60.785000,-161.839996,85000.0,30000060066,"BETHEL/BETHEL AIRPORT, AK",2021-05-31 12:00:00,nnnnnnnnnnn
395960,0000395960,0.769,60.785000,-161.839996,92500.0,30000060066,"BETHEL/BETHEL AIRPORT, AK",2021-05-31 12:00:00,nnnnnnnnnnn


In [10]:
dd

,bias_estimate,hur,hur_bias_estimate,hur_breakpoints,hur_snht_test,lat,lon,obs_minus_bg,plev,report_id,station_id,time,trajectory_label
3,NaN,0.660,0.0,0,0.0,60.779999,-161.720001,NaN,50000.0,60000000001,"BETHEL/BETHEL AIRPORT, AK",1948-01-01 15:00:00,nnnnnnnnnnn
4,NaN,0.750,0.0,0,0.0,60.779999,-161.720001,NaN,70000.0,60000000001,"BETHEL/BETHEL AIRPORT, AK",1948-01-01 15:00:00,nnnnnnnnnnn
5,NaN,0.730,0.0,0,0.0,60.779999,-161.720001,NaN,85000.0,60000000001,"BETHEL/BETHEL AIRPORT, AK",1948-01-01 15:00:00,nnnnnnnnnnn
9,NaN,0.440,0.0,0,0.0,60.779999,-161.720001,NaN,50000.0,60000000003,"BETHEL/BETHEL AIRPORT, AK",1948-01-02 15:00:00,nnnnnnnnnnn
10,NaN,0.690,0.0,0,0.0,60.779999,-161.720001,NaN,70000.0,60000000003,"BETHEL/BETHEL AIRPORT, AK",1948-01-02 15:00:00,nnnnnnnnnnn
...,...,...,...,...,...,...,...,...,...,...,...,...,...
395957,NaN,0.655,0.0,0,0.0,60.785000,-161.839996,NaN,50000.0,30000060066,"BETHEL/BETHEL AIRPORT, AK",2021-05-31 12:00:00,nnnnnnnnnnn
395958,NaN,0.524,0.0,0,0.0,60.785000,-161.839996,NaN,70000.0,30000060066,"BETHEL/BETHEL AIRPORT, AK",2021-05-31 12:00:00,nnnnnnnnnnn
395959,NaN,0.842,0.0,0,0.0,60.785000,-161.839996,NaN,85000.0,30000060066,"BETHEL/BETHEL AIRPORT, AK",2021-05-31 12:00:00,nnnnnnnnnnn
395960,NaN,0.769,0.0,0,0.0,60.785000,-161.839996,NaN,92500.0,30000060066,"BETHEL/BETHEL AIRPORT, AK",2021-05-31 12:00:00,nnnnnnnnnnn


In [14]:
plev = 100
cdata = dcd[dcd.plev == plev*100]
adata = dd[dd.plev == plev*100]
# fig, ax = plt.subplots()
# ax.scatter(cdata['time'], cdata.humidity_bias_estimate, label='CDS', s=50)
# ax.scatter(adata['time'], adata.hur_bias_estimate, label='Calc', s=20)

# ax.set(xlabel='time', ylabel='Adjustment: '+adj,
#        title= str(station)+' Humidity '+str(plev)+' hPa '+time)
# ax.legend()
# ax.grid()

# if save:
#     fig.savefig(str(station)+'_Humidity_'+str(plev)+'_hPa_'+time+'.png')
# plt.show()
# return

In [18]:
adata.hur_bias_estimate

52617     0.25103
93069    -0.00000
117428    0.25103
120598    0.25103
120655   -0.00000
           ...   
395799    0.00000
395831    0.00000
395891    0.00000
395921    0.00000
395951    0.00000
Name: hur_bias_estimate, Length: 8432, dtype: float32

In [7]:
plt_adj(dcd, dd, 100, station, 'day', 'humidity_bias_estimate', save=True)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3418, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-5f4f52e9ac30>", line 1, in <module>
    plt_adj(dcd, dd, 100, station, 'day', 'humidity_bias_estimate', save=True)
  File "<ipython-input-3-9c9412e389ac>", line 14, in plt_adj
    fig.savefig(str(station)+'_Humidity_'+str(plev)+'_hPa_'+time+'.png')
  File "/opt/anaconda3/lib/python3.7/site-packages/matplotlib/figure.py", line 2311, in savefig
    self.canvas.print_figure(fname, **kwargs)
  File "/opt/anaconda3/lib/python3.7/site-packages/matplotlib/backend_bases.py", line 2217, in print_figure
    **kwargs)
  File "/opt/anaconda3/lib/python3.7/site-packages/matplotlib/backend_bases.py", line 1639, in wrapper
    return func(*args, **kwargs)
  File "/opt/anaconda3/lib/python3.7/site-packages/matplotlib/backends/backend_agg.py", line 509, in print_png
    FigureCanvasA

TypeError: object of type 'NoneType' has no len()

Error in callback <function flush_figures at 0x7f82b8376b90> (for post_execute):


KeyboardInterrupt: 

In [ ]:
plt_adj(ncd, nd, 100, station, 'night', 'humidity_bias_estimate', save=True)
plt_adj(dcd, dd, 500, station, 'day', 'humidity_bias_estimate', save=True)
plt_adj(ncd, nd, 500, station, 'night', 'humidity_bias_estimate', save=True)
plt_adj(dcd, dd, 850, station, 'day', 'humidity_bias_estimate', save=True)
plt_adj(ncd, nd, 850, station, 'night', 'humidity_bias_estimate', save=True)

In [ ]:
station = '91413'
dd, nd, dcd, ncd = get_adj(station)

plt_adj(dcd, dd, 100, station, 'day', 'humidity_bias_estimate', save=True)
plt_adj(ncd, nd, 100, station, 'night', 'humidity_bias_estimate', save=True)
plt_adj(dcd, dd, 500, station, 'day', 'humidity_bias_estimate', save=True)
plt_adj(ncd, nd, 500, station, 'night', 'humidity_bias_estimate', save=True)
plt_adj(dcd, dd, 850, station, 'day', 'humidity_bias_estimate', save=True)
plt_adj(ncd, nd, 850, station, 'night', 'humidity_bias_estimate', save=True)

In [ ]:
station = '11035'
dd, nd, dcd, ncd = get_adj(station)

plt_adj(dcd, dd, 100, station, 'day', 'humidity_bias_estimate', save=True)
plt_adj(ncd, nd, 100, station, 'night', 'humidity_bias_estimate', save=True)
plt_adj(dcd, dd, 500, station, 'day', 'humidity_bias_estimate', save=True)
plt_adj(ncd, nd, 500, station, 'night', 'humidity_bias_estimate', save=True)
plt_adj(dcd, dd, 850, station, 'day', 'humidity_bias_estimate', save=True)
plt_adj(ncd, nd, 850, station, 'night', 'humidity_bias_estimate', save=True)

In [ ]:
station = '10393'
dd, nd, dcd, ncd = get_adj(station)

plt_adj(dcd, dd, 100, station, 'day', 'humidity_bias_estimate', save=True)
plt_adj(ncd, nd, 100, station, 'night', 'humidity_bias_estimate', save=True)
plt_adj(dcd, dd, 500, station, 'day', 'humidity_bias_estimate', save=True)
plt_adj(ncd, nd, 500, station, 'night', 'humidity_bias_estimate', save=True)
plt_adj(dcd, dd, 850, station, 'day', 'humidity_bias_estimate', save=True)
plt_adj(ncd, nd, 850, station, 'night', 'humidity_bias_estimate', save=True)